In [1]:
from bs4 import BeautifulSoup
import os
import pickle
from selenium import webdriver

In [2]:
url = "http://metropolitan-market.com/specials/search.php?c%5B%5D=all"

In [3]:
driver_file = os.environ.get("SPECIALS_DRIVER")

In [4]:
file = "testCookies.pkl"

In [5]:
soup_format = 'lxml'

In [6]:
def make_browser(driver_file):
    browser = webdriver.Chrome(driver_file)
    return browser

In [7]:
def store_cookies(browser, url, file):
    browser.get(url)
    pickle.dump( browser.get_cookies() , open(file,"wb"))

In [8]:
def get_cookies(file):
    cookies = pickle.load(open(file, "rb"))
    return cookies

In [9]:
def access_site_innerHTML(browser, url, cookies):
    for cookie in cookies:
        browser.add_cookie(cookie)
    browser.get(url)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    return innerHTML

In [10]:
def make_soup(innerHTML, soup_format):
    bs = BeautifulSoup(innerHTML, soup_format)
    return bs

In [11]:
def get_specials(bso):
    specials = bso.find("div", {"id": "specialsContent"})
    return specials

In [12]:
def get_items(specials):
    items = specials.findAll("div", {"class": "item_view"})
    return items

In [13]:
def parse_specials_to_dict(items):
    ws = {}
    l = 'local'
    o = 'organic'
    for item in items:
        id = item.get("id")
        price = item.find("strong", {"class": "price"})
        sm = item.findAll("small")
        brand = sm[0].text
    #     description = sm[1].text
        deets = item.find("h4")
        name = ''.join(deets.find('br').next_siblings).strip()
        ws[id] = {}
        ws[id]['name'] = name
        ws[id]['price'] = price.text
    #     ws[id]['description'] = description
        if l in brand or l.upper() in brand:
            ws[id]['local'] = True
            ws[id]['brand'] = brand[6:]
        else:
            ws[id]['local'] = False
            ws[id]['brand'] = brand
        if o in brand or o.upper() in brand or o in name or o.upper() in name or o in name.lower():
            ws[id]['organic'] = True
        else:
            ws[id]['organic'] = False
    return ws

In [14]:
b = make_browser(driver_file)

In [15]:
store_cookies(b, url, file)

In [16]:
cookies = get_cookies(file)

In [17]:
innerHTML = access_site_innerHTML(b, url, cookies)

In [18]:
bso = make_soup(innerHTML, soup_format)

In [19]:
specials = get_specials(bso)

In [20]:
items = get_items(specials)

In [21]:
ws = parse_specials_to_dict(items)

In [22]:
print(ws)

{'A1': {'name': 'Extra Lean Ground Beef', 'price': '$4.99 lb.', 'local': True, 'brand': 'Naturally Raised', 'organic': False}, 'A2': {'name': 'Rioja Crianza', 'price': '$11.99', 'local': False, 'brand': 'Vina Hermenia', 'organic': False}, 'A3': {'name': 'Free Range Chicken Wings', 'price': '$1.99 lb.', 'local': False, 'brand': 'Naturally Raised', 'organic': False}, 'A4': {'name': 'Beef Top Sirloin Steak', 'price': '$8.99', 'local': True, 'brand': 'Naturally Raised', 'organic': False}, 'A5': {'name': 'True Cod Fillets', 'price': '$6.99 lb.', 'local': False, 'brand': 'Wild Caught', 'organic': False}, 'A6': {'name': 'Lobster Tail', 'price': '$10.99', 'local': False, 'brand': 'Wild, Maine', 'organic': False}, 'A7': {'name': 'Fuji Apples', 'price': '$1.99 lb.', 'local': True, 'brand': 'ORGANIC', 'organic': True}, 'A8': {'name': 'Strawberries', 'price': '$3.99', 'local': False, 'brand': 'Fresh', 'organic': False}, 'A9': {'name': 'Extra Mature Scottish Cheddar', 'price': '$5.99', 'local': Fal

In [23]:
print(len(ws))

156


In [25]:
dates = specials.find("div", {"class": "specials_dates"})
print(dates.text.strip())

Prices Valid Wed, September 20th — Tue, October 3rd.
